### Model Training and Cross Validation

In [1]:
#load library
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Subset
#load models and other customized component
from models.base_model import BaseModel, MagicModel1, MagicModel2
from utils.model_util import fit, predict, hyp_random_search
from utils.loss_functions import ProbCatCrossEntropyLoss
from utils.performance import categorical_accuracy
from utils.preprocessing import GlobalContrastNormalization, ZCATransformation, zca_whitening_matrix
from utils.data import FaceDataset

import pickle

In [2]:
data = FaceDataset('data/face_data/', training=True)

In [3]:
#set norm zca transform
#calculate ZCA whitening
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.ToPILImage(),
     transforms.Resize(32),
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

In [4]:
#get data (_, 1, 16, 16), resize to (_, 1, 32, 32)
trainset = FaceDataset('data/face_data/', training=True, transform=transform)
testset = FaceDataset('data/face_data/', training=False, transform=transform)

In [5]:
#split validation set from training set
smallset = Subset(trainset, np.arange(200))

train_length = len(trainset)
train_index = np.arange(int(train_length*0.8))
valid_index = np.arange(int(train_length*0.8), train_length)

validset = Subset(trainset, valid_index)
trainset = Subset(trainset, train_index)

smallloader = DataLoader(smallset, batch_size = 4, shuffle=False, num_workers=4, pin_memory =True)
trainloader = DataLoader(trainset, batch_size=250, shuffle=True, num_workers=4, pin_memory =True)
validloader = DataLoader(validset, batch_size=250, shuffle=False, num_workers=4, pin_memory =True)
testloader = DataLoader(testset, batch_size=250, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
#initialize model, optimizer, etc
model = MagicModel1((1, 32, 32), 10)
loss_function = ProbCatCrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3) #, momentum=0.9
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 10, gamma=1)

In [7]:
#training
history = fit(model, trainloader, optimizer, loss_function, lr_scheduler, epochs=10, valid_loader = validloader, measure = ['accuracy'], verbose = 1)
#history = fit(model, smallloader, optimizer, loss_function, lr_scheduler, epochs=40, measure = ['accuracy'])

Epoch 0:   4% 1/24 [00:00<00:13,  1.65it/s, training_loss=2.3, train_accuracy=0.016]

/home/jwang75/tmp/workspace/models/base_model.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


Epoch 0: 100% 24/24 [00:04<00:00,  5.32it/s, training_loss=0.329, train_accuracy=0.896, val_loss=0.159, val_accuracy=0.972]
Epoch 1: 100% 24/24 [00:04<00:00,  5.66it/s, training_loss=0.0274, train_accuracy=0.99, val_loss=0.0243, val_accuracy=0.992]
Epoch 2: 100% 24/24 [00:04<00:00,  5.69it/s, training_loss=0.0153, train_accuracy=0.995, val_loss=0.0123, val_accuracy=0.997]
Epoch 3: 100% 24/24 [00:04<00:00,  5.67it/s, training_loss=0.00819, train_accuracy=0.998, val_loss=0.00834, val_accuracy=0.997]
Epoch 4: 100% 24/24 [00:04<00:00,  5.63it/s, training_loss=0.00338, train_accuracy=1, val_loss=0.00806, val_accuracy=0.996]
Epoch 5: 100% 24/24 [00:04<00:00,  5.65it/s, training_loss=0.00223, train_accuracy=1, val_loss=0.00495, val_accuracy=0.996]
Epoch 6: 100% 24/24 [00:04<00:00,  5.64it/s, training_loss=0.00129, train_accuracy=1, val_loss=0.00892, val_accuracy=0.997]
Epoch 7: 100% 24/24 [00:04<00:00,  5.64it/s, training_loss=0.00118, train_accuracy=1, val_loss=0.00761, val_accuracy=0.996]
E

#plot learning curve for loss and accuracy
file_name = "norm_zca_relu_model"
epochs = np.arange(50)
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle("Normalization ZCA relu Model")
ax[0].plot(epochs, history['training history'], linestyle='-', color='b', label='training')
ax[0].plot(epochs, history['validation history'], linestyle='-', color='r', label='validation')
ax[0].set_xlabel("epoch")
ax[0].set_ylabel("loss")

ax[1].plot(epochs, history['training accuracy'], linestyle='-', color='b', label='training')
ax[1].plot(epochs, history['validation accuracy'], linestyle='-', color='r', label='validation')
ax[1].set_xlabel("epoch")
ax[1].set_ylabel("accuracy")

ax[1].legend(loc='lower right')
fig.savefig(file_name+".png", format='png', bbox_inches = 'tight', pad_inches=0.05)

In [8]:
#testing
test_predict = predict(model, testloader)

In [9]:
test_labels = np.concatenate([yb for _, yb in testloader])
categorical_accuracy(np.argmax(test_predict, axis=1).flatten(), test_labels)

0.9978729061419835